In [2]:
%reset -f
%load_ext autoreload
%autoreload 2

import numpy as np
from matplotlib import pyplot as plt

#known information
takeoff_soc = 0.9
cell_capacity = 3.55 #amp hours
takeoff_time = 1 / 60 #in minutes
takeoff_power = 290e3 #in W
cruise_time = 35 / 60 
cruise_power = 75e3
abort_time = 1 / 60
abort_power = 180e3
reserve_time = 5 / 60
reserve_power = 75e3
descent_time = 1 / 60
descent_power = 150e3
start_charge = 0.9
pack_capacity = 24 * 2 * cell_capacity

def solve_problem(current_soc, current_power, event_time):
    resistance = (0.015 * current_soc**2 - 0.025 * current_soc + 0.104)
    ocv = (0.39 * current_soc**2 + 0.07 * current_soc + 3.7) * 1e3
    current = (ocv/(2*resistance)) - np.sqrt((ocv/(2*resistance))**2 - current_power/resistance)
    battery_voltage = ocv - current * resistance
    efficiency = battery_voltage / ocv
    power_used = current * event_time
    new_soc = (pack_capacity * current_soc - power_used) / pack_capacity
    out = {'resistance':resistance,'ocv':ocv,'current':current,'battery_voltage':battery_voltage,'efficiency':efficiency,'power_used':power_used,'new_soc':new_soc}
    return out
    

takeoff_out = solve_problem(takeoff_soc, takeoff_power, takeoff_time)
cruise_out = solve_problem(takeoff_out['new_soc'], cruise_power, cruise_time)
abort_out = solve_problem(cruise_out['new_soc'], abort_power, abort_time)
reserve_out = solve_problem(abort_out['new_soc'], reserve_power, reserve_time)
descent_out = solve_problem(reserve_out['new_soc'], descent_power, descent_time)

print(f"soc after takeoff: {takeoff_out['new_soc']}")
print(f"soc after cruise: {cruise_out['new_soc']}")
print(f"soc after abort: {abort_out['new_soc']}")
print(f"soc after reserve: {reserve_out['new_soc']}")
print(f"soc after descent: {descent_out['new_soc']}")


# pack_capacity = 24 * 2 * cell_capacity

# takeoff_r = calculate_r(takeoff_soc)
# takeoff_ocv = calculate_ocv(takeoff_soc)
# takeoff_current = calculate_current(takeoff_r,takeoff_ocv,takeoff_power)
# takeoff_battery_voltage = takeoff_ocv - takeoff_current * takeoff_r
# takeoff_efficiency = takeoff_battery_voltage / takeoff_ocv
# takeoff_power_used = takeoff_current * takeoff_time
# takeoff_end_soc = (pack_capacity * takeoff_soc - takeoff_power_used) / pack_capacity



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
soc after takeoff: 0.8930346206799535
soc after cruise: 0.8299795199630249
soc after abort: 0.8256028056756493
soc after reserve: 0.8164830869415116
soc after descent: 0.8128277183373028
